In [1]:
%pip install -q "unstructured[csv,xlsx]" "astrapy" "openai" "numpy" "cassandra-driver" "python-dotenv" "html2text"


Note: you may need to restart the kernel to use updated packages.


In [2]:
!ls -R "data/01 SMART Methane Project Registration Samples"


BPX Gas Quality Analysis - 20190723.xlsx
BPX Haynesville Emission Workbook 20190919.xlsx
BPX Haynesville Facility P_ID.pdf
BPX Haynesville OGMP-2.0-Upstream_Reporting_Templat_MOCK.xlsx
Production Data
pdf-images

data/01 SMART Methane Project Registration Samples/Production Data:
BPX Monthly Gas Sums.csv BPX daily total vol.csv

data/01 SMART Methane Project Registration Samples/pdf-images:
BPX Haynesville Facility P_ID.jpg      BPX Haynesville Facility P_ID_6.jpg
BPX Haynesville Facility P_ID_2.jpg    BPX Haynesville Facility P_ID_7.jpg
BPX Haynesville Facility P_ID_3.jpg    BPX Haynesville Facility P_ID_8.jpg
BPX Haynesville Facility P_ID_4.jpg    BPX Haynesville Facility P_ID_9(1).jpg
BPX Haynesville Facility P_ID_5.jpg    BPX Haynesville Facility P_ID_9.jpg


In [3]:
import os
from dotenv import load_dotenv  
load_dotenv()


True

In [4]:
import glob
files = glob.glob("data/01 SMART Methane Project Registration Samples/*.xlsx")


In [5]:
from unstructured.partition.auto import partition
import html2text
import json

def to_astradb(el):
    return {
        "source": el.metadata.filename,
        "page": el.metadata.page_name,
        "number": el.metadata.page_number,
        "text": el.text.replace("'", ""),
        "render": html2text.html2text(el.metadata.text_as_html.replace("'",""))
    }
    
print("Starting to parse data: ", end="")
jsons = []
for file in files:
    for element in partition(filename=file):
        print(".", end="", flush=True)
        if element.metadata.text_as_html is not None:
            jsons.append(to_astradb(element))
            
print("Finished parsing data", end="")


Starting to parse data: ........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................Finished parsing data

In [6]:
import pandas as pd

df = pd.DataFrame.from_dict(jsons, orient='columns')
df = df.dropna()
cols = ['source', 'page', 'render']
df["combined"] = df[cols].apply(lambda row: '\n'.join(row.values.astype(str)), axis=1)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   source    61 non-null     object
 1   page      61 non-null     object
 2   number    61 non-null     int64 
 3   text      61 non-null     object
 4   render    61 non-null     object
 5   combined  61 non-null     object
dtypes: int64(1), object(5)
memory usage: 3.0+ KB


In [7]:
df.sample(5)


,source,page,number,text,render,combined
37,BPX Haynesville Emission Workbook 20190919.xlsx,BPX EPA CO2ef,6,\n\n\n1\nWalker 1H – single\n\n\n1\nTaz 1H – s...,1 | Walker 1H – single \n---|--- \n1 | Taz 1...,BPX Haynesville Emission Workbook 20190919.xls...
53,BPX Haynesville Emission Workbook 20190919.xlsx,BPX CO2c,8,\n\n\nEPA Combustion factors\n\n\nCooler Engin...,EPA Combustion factors | | | Cooler Engine F...,BPX Haynesville Emission Workbook 20190919.xls...
47,BPX Haynesville Emission Workbook 20190919.xlsx,CH4 Emissions (Other),7,\n\n\nEmission Source\nUnits\nComments\n\n\n,Emission Source | Units | Comments \n---|---|...,BPX Haynesville Emission Workbook 20190919.xls...
38,BPX Haynesville Emission Workbook 20190919.xlsx,BPX EPA CO2ef,6,\n\n\nMethane\nValue\n\n\nscf/pound (6)\n28.90...,Methane | Value \n---|--- \nscf/pound (6) | ...,BPX Haynesville Emission Workbook 20190919.xls...
11,BPX Haynesville Emission Workbook 20190919.xlsx,BPX Total PoC Prod,2,\n\n\nWell Name\nMean Daily Production\n\n\n\n...,Well Name | Mean Daily Production | | | | ...,BPX Haynesville Emission Workbook 20190919.xls...


In [8]:
for a,b in df.iterrows():
    print("----------------------------- New Table ------------------------")
    print (b.combined.replace('\n\n',' '))


----------------------------- New Table ------------------------
BPX Haynesville OGMP-2.0-Upstream_Reporting_Templat_MOCK.xlsx
L1 and L2 Assets
|  |  |  |  |  |  |  |  |  |  | L1 - Emissions reported for a venture at asset or country level | L2 - Emissions reported in consolidated, simplified sources categories |  |  |  |  |  |  |  | Comments on variation from prior year | Investments/divestments of facilities in the asset (Yes/No)  
---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---  
Asset/ Venture Name | Asset id | Type of Asset | Description of the Asset | Country | Latitude | Longitude | Operated? | Operator | % Equity \n (Note- Ownership of emissions) | Specify type of equity | Total CH4 emissions, metric tonnes | Stationary Combustion, metric tonnes CH4 | Flaring (Incomplete Combustion), metric tonnes CH4 | Fugitives, metric tonnes CH4 | Venting , metric tonnes CH4 | Others - Specify in the comment section | Total, metric tonnes CH4 | Comment

### Connect via CQL

In [9]:

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# Load secrets from JSON file
with open("../../demos/astradb-connections/pacp/pacp-token.json") as f:
    secrets = json.load(f)

# Extract credentials from secrets
id = secrets.get("clientId")
secret = secrets.get("secret")

# Configure cloud connection
cloud_config = {
    'secure_connect_bundle': "../../demos/astradb-connections/pacp/secure-connect-pacp.zip"
}

# Create authentication provider
auth_provider = PlainTextAuthProvider(id, secret)

# Connect to Cassandra cluster
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
# Create session
session = cluster.connect()
# Execute query
row = session.execute("select release_version from system.local").one()
if row:
    print(row[0])
    a = session
else:
    print("An error occurred.")


4.0.11-09ec37c912ed


### OpenAI

In [10]:
from openai import Client
client = Client(api_key=os.environ.get("OPENAI_API_KEY"))


### Embedding via Data API

In [11]:
import numpy as np

BATCH_SIZE = 20
MODEL_NAME = "text-embedding-3-small"

print("Starting to vectorize data ", end="")
vectors = []

# Loop BATCH_SIZE at the time
for k,g in df.groupby(np.arange(len(df))//BATCH_SIZE):
    # compute the embedding vectors for this batch    
    for vector in client.embeddings.create(input=g.combined, model=MODEL_NAME).data:
        vectors.append(vector.embedding)
        print(".", end="", flush=True)

vector_series  = pd.Series(vectors, name="vector")
print("Finished vectorizing entries.")


Starting to vectorize data .............................................................Finished vectorizing entries.


In [12]:
vector_series.shape


(61,)

In [13]:
cql = "use default_keyspace;"
session.execute(cql)


In [14]:
cql = """CREATE TABLE IF NOT EXISTS well_data (
        id INT PRIMARY KEY,
        source TEXT,
        page TEXT,
        number INT,
        text TEXT,
        render TEXT,
        vector VECTOR <FLOAT, 1536>
);"""
session.execute(cql)


In [15]:
cql = """CREATE INDEX IF NOT EXISTS vector_index ON well_data(vector);"""
session.execute(cql)


In [16]:
cql = "truncate table well_data"
session.execute(cql)


In [17]:
insert_statements = []
for line, vector in zip(df.iterrows(), vectors):
    indx = line[0]
    row = line[1]
    insert_statement = f"""INSERT INTO well_data (id, source, page, number, text, render, vector) VALUES ({indx}, '{row['source']}', '{row['page']}', {row['number']}, '{row['text']}', '{row['render']}', {vector});"""
    insert_statements.append(insert_statement)


In [18]:
insert_statement[:100]


"INSERT INTO well_data (id, source, page, number, text, render, vector) VALUES (60, 'BPX Gas Quality "

In [19]:
for insert_statement in insert_statements:
    print(insert_statement[:100])
    session.execute(insert_statement)


INSERT INTO well_data (id, source, page, number, text, render, vector) VALUES (0, 'BPX Haynesville O
INSERT INTO well_data (id, source, page, number, text, render, vector) VALUES (1, 'BPX Haynesville O
INSERT INTO well_data (id, source, page, number, text, render, vector) VALUES (2, 'BPX Haynesville O
INSERT INTO well_data (id, source, page, number, text, render, vector) VALUES (3, 'BPX Haynesville O
INSERT INTO well_data (id, source, page, number, text, render, vector) VALUES (4, 'BPX Haynesville O
INSERT INTO well_data (id, source, page, number, text, render, vector) VALUES (5, 'BPX Haynesville O
INSERT INTO well_data (id, source, page, number, text, render, vector) VALUES (6, 'BPX Haynesville E
INSERT INTO well_data (id, source, page, number, text, render, vector) VALUES (7, 'BPX Haynesville E
INSERT INTO well_data (id, source, page, number, text, render, vector) VALUES (8, 'BPX Haynesville E
INSERT INTO well_data (id, source, page, number, text, render, vector) VALUES (9, 'BPX Hayn

### Prompting

In [20]:
persona = """You are an expert analyst in Oil and Gas Industry with focus on Environmental Impact Analysis. You have intense knowledge of how well production works. you are extremely familiar with the operations, setup, machines and how to calculate output etc of these wells.
As part of this job you are examining the environmental impact of well operations on the surrounding environment. You are particularly interested in assessing factors such as air and water pollution, habitat disruption, and overall ecosystem health.

To conduct this analysis, you will be supplied with actual raw production numbers from well. based on this data please answer user question. and please try to be as elaborative as possible and explain your reasoning.
"""


In [23]:
question = """Tell me the factor used to convert produced methane volume (MCF) to produced energy (MMBtu) from Bandicoot 1hb"""
question_vector = client.embeddings.create(input=question,model=MODEL_NAME)

cql = """SELECT  text, render, similarity_cosine(vector, {vector})
    FROM well_data
    ORDER BY vector ANN OF {vector} LIMIT 10;
""".format(vector=question_vector.data[0].embedding)
rows = session.execute(cql)
context = '\n'.join(html2text.html2text(row.text) for row in rows)
print(tables)


Well Name Mean Daily Production mcfd Constants Carbon Intensity GHG Intensity
Bandicoot 1hb 4081.758 pounds / metric tonne 2204.62 Total Volume Total Volume
Bandicoot 2h 8274.224011 scf co2 / pound 8.741 mcf 3279.205835 mcf 49.582404
Pluto 1h 9523.823398 GWP 29 Tonnes 584.430899 Tonnes 416.837455 Pluto 1hb
8864.445125 produced water 78628.09 bbls Volume per mmBTU Volume per mmBTU
T-Rex 1h 12909.733735 scf/pound 28.9107 mcf 0.040742 mcf 0.000616 Taz 1h
9147.73176 scf/TeCH4 52158 Tonnes 0.007261 Tonnes 0.005179 Taz 2h 11747.526401
TeCO2/MBOE 0.802444 TeCO2/MBOE 0.075087 Taz 3h 13511.448622 Walker J1h
3241.835531 Total 81302.52658


Well Name Mean Daily Production mcfd Constants Carbon Intensity GHG Intensity
Bandicoot 1hb pounds / metric tonne 2204.62 Total Volume Total Volume
Bandicoot 2h scf co2 / pound 8.741 mcf 196494.0157 mcf 2716.593758 Pluto 1h
GWP 29 Tonnes 35052.44829 Tonnes 24996.84085 Pluto 1hb produced water 78628.09
bbls Volume per mmBTU Volume per mmBTU T-Rex 1h scf/pound 2

In [24]:
print(client.chat.completions.create(
        messages=[
            {"role": "system", "content": persona},
            {"role": "user", "content": context},
            {"role": "user", "content": question}
        ],
        model="gpt-4",
        max_tokens=1000,
        temperature=0.1
    ).choices[0].message.content)


The factor used to convert produced methane volume (MCF) to produced energy (MMBtu) is the energy content of methane. Methane has an energy content of approximately 1,031 BTU per cubic foot (BTU/cf). 

To convert this to MMBtu (million BTU) per MCF (thousand cubic feet), we divide by 1,000,000 (to convert BTU to MMBtu) and multiply by 1,000 (to convert cubic feet to MCF). 

So, the conversion factor is (1031 BTU/cf) * (1 MMBtu / 1,000,000 BTU) * (1,000 cf / 1 MCF) = 1.031 MMBtu/MCF.

This means that for every thousand cubic feet (MCF) of methane produced from the Bandicoot 1hb well, we get approximately 1.031 million BTU (MMBtu) of energy.
